In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
width = 28
height = 28
total_pixels = width * height
num_class = 10

In [3]:
x = tf.placeholder(tf.float32, [None, total_pixels])
y = tf.placeholder(tf.float32, [None, num_class])

In [4]:
x_image = tf.reshape(x, [-1, 28,28,1])
W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32], stddev = 0.1 ))
b_conv1 = tf.Variable(tf.constant(0.1, shape = [32]))     
conv1 = tf.nn.conv2d(x_image, W_conv1, strides = [1,1,1,1],
                     padding = 'SAME') + b_conv1
raw_conv1 = tf.nn.relu(conv1)
conv1 = tf.nn.max_pool(raw_conv1, ksize = [1,2,2,1], 
                       strides = [1,2,2,1],padding = 'SAME')


In [5]:
W_conv2 = tf.Variable(tf.truncated_normal([5,5,32,64], stddev = 0.1 ))
b_conv2 = tf.Variable(tf.constant(0.1, shape = [64])) 
conv2 = tf.nn.conv2d(conv1, W_conv2, strides = [1,1,1,1],
                     padding = 'SAME') + b_conv2
raw_conv2 = tf.nn.relu(conv2)
conv2 = tf.nn.max_pool(raw_conv2, ksize = [1,2,2,1], 
                       strides = [1,2,2,1], padding = 'SAME')


In [6]:
layer_fully = tf.reshape(conv2, [-1, 7*7*64])
W_fc1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev = 0.1 ))
b_fc1 =  tf.Variable(tf.constant(0.1, shape = [1024])) 
fc1 = tf.matmul(layer_fully, W_fc1) + b_fc1
fc1 = tf.nn.relu(fc1)

In [7]:
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(fc1, keep_prob)

In [8]:
W_fc2 = tf.Variable(tf.truncated_normal([1024, num_class], stddev = 0.1 ))
b_fc2 =  tf.Variable(tf.constant(0.1, shape = [num_class])) 
fc2 = tf.matmul(layer_drop, W_fc2) + b_fc2
fc2 = tf.nn.relu(fc2)
y_pred = tf.nn.softmax(fc2)

In [9]:
loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices = [1]))
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_pred = tf.equal(tf.argmax(y ,1), tf.argmax(y_pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
init = tf.global_variables_initializer()

In [10]:
with tf.device('/device:GPU:0'):
    with tf.Session() as sess:
        sess.run(init)
        for i in range(2000):
            batch_x, batch_y = mnist.train.next_batch(50)
            if i%100 == 0:
                train_acc = accuracy.eval(feed_dict = {x: batch_x,
                                         y: batch_y, keep_prob: 1.0})
                print('step: %d, training accuracy %g' %(i, train_acc))
            optimizer.run(feed_dict = {x: batch_x,
                                         y: batch_y, keep_prob: 0.5}) 
        n_batches = mnist.test.images.shape[0]
        sum_acc = 0.0
        for i in range(n_batches):
            batch_x, batch_y = mnist.test.next.batch(50)
            sum_acc = sum_acc + accuracy.eval(feed_dict = {x: batch_x,
                                         y: batch_y, keep_prob: 1.0})
        print('test accuracy {}'.format(sum_acc/n_batches))
        
        

step: 0, training accuracy 0.08
step: 100, training accuracy 0.06
step: 200, training accuracy 0.2
step: 300, training accuracy 0.26
step: 400, training accuracy 0.32
step: 500, training accuracy 0.42
step: 600, training accuracy 0.7
step: 700, training accuracy 0.88
step: 800, training accuracy 0.94
step: 900, training accuracy 0.9
step: 1000, training accuracy 1
step: 1100, training accuracy 0.98
step: 1200, training accuracy 0.96
step: 1300, training accuracy 0.94
step: 1400, training accuracy 0.96
step: 1500, training accuracy 0.96
step: 1600, training accuracy 0.96
step: 1700, training accuracy 1
step: 1800, training accuracy 0.96
step: 1900, training accuracy 0.98
step: 2000, training accuracy 0.94
step: 2100, training accuracy 0.96
step: 2200, training accuracy 0.98
step: 2300, training accuracy 0.98
step: 2400, training accuracy 0.98
step: 2500, training accuracy 1
step: 2600, training accuracy 0.96
step: 2700, training accuracy 1
step: 2800, training accuracy 1
step: 2900, tra

KeyboardInterrupt: 

In [16]:
with tf.Session() as sess:
    kernels = sess.run(tf.reshape(tf.transpose(W_conv1, perm = [2,3,0,1]), [32, -1]))
    
!wget --output-document utils1.py http://deeplearning.net/tutorial/code/utils.py
import utils1
from utils1 import tile_raster_images
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
image = Image.fromarray(tile_raster_image(kernels, img_shape = (5,5),
                                         tile_shape = (4,8), tile_spacing = (1,1)))
plt.rcParams['figure.figsize'] = (18.0,18.0)
imgplot = plt.imshow(image)
imgplot.set_cmap('gray')

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[node Variable/read (defined at <ipython-input-4-c6ccb9f320e2>:2)  = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Variable)]]
	 [[{{node Reshape_5/_1}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_11_Reshape_5", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Variable/read', defined at:
  File "D:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\appl